## INTRO



In [1]:
from google.colab import drive
import os

import pandas as pd
import numpy as np

drive.mount('/content/gdrive')
os.chdir('/content/gdrive/My Drive/USP/2024_2º/MQA/')

pd.set_option('display.max_rows', 6)
pd.set_option('display.max_columns', None)

Mounted at /content/gdrive


## READ_CSV

In [31]:
df = pd.read_csv('atuacoes_2.csv', encoding='UTF-8', nrows=500) #, dtype=str, nrows=1000)

df['ANO-FIM'] = df['ANO-FIM'].fillna(2024)
df['FLAG-DEDICACAO-EXCLUSIVA'] = df['FLAG-DEDICACAO-EXCLUSIVA'].replace('NAO', False).replace('SIM', True)

df = df.astype({
    'NRO-ID-CNPQ': int,
    'CODIGO-INSTITUICAO': str,
    'NOME-INSTITUICAO': str,
    'ANO-INICIO': int,
    'ANO-FIM': int,
    'ENQUADRAMENTO-FUNCIONAL': str,
    'TIPO-DE-VINCULO': str,
    'OUTRAS-INFORMACOES': str,
    'CARGA-HORARIA-SEMANAL': float,
    'FLAG-DEDICACAO-EXCLUSIVA': bool
})

df = df.fillna(' ')
df2 = df.copy(deep=True)

#df

In [32]:
# LISTA DE IES NACIONAIS
# https://dadosabertos.mec.gov.br/indicadores-sobre-ensino-superior/item/181-instituicoes-de-educacao-superior-do-brasil
# https://dadosabertos.mec.gov.br/images/conteudo/Ind-ensino-superior/2022/PDA_Lista_Instituicoes_Ensino_Superior_do_Brasil_EMEC.csv

df_uni = pd.read_csv('sup.csv', encoding='UTF-8')
df_uni_filtered = df_uni[['NOME_DA_IES', 'SIGLA']]
df_uni_filtered = df_uni_filtered.fillna('')
df_uni_filtered = df_uni_filtered.sort_values(['NOME_DA_IES'], ascending=False)
df_uni_filtered

,NOME_DA_IES,SIGLA
862,iCEV - Instituto de Ensino Superior,
2205,VOAS - INSTITUTO DE APRENDIZAGEM SUPERIOR,VOAS
2873,Universidade do Estado de Mato Grosso Carlos A...,UNEMAT
334,Universidade Universus Veritas Guarulhos,Univeritas UNG
4124,Universidade Regional do Cariri,
...,...,...
2416,ACADEMIA JUDICIAL DO TRIBUNAL DE JUSTIÇA DE SA...,AJ
4234,ACADEMIA DE POLÍCIA MILITAR DO ESPÍRITO SANTO,APM/ES
30,ACADEMIA DE POLÍCIA MILITAR DE MINAS GERAIS,APM
3933,ACADEMIA DA FORÇA AÉREA,AFA


## PRE-PROCESSAMENTO

In [33]:
def clean_names(df, column_name):

    # Remover '&'
    df[column_name] = df[column_name].replace('&AMP;', ' ', regex=True)

    # Remover todos os caracteres nao alfa-numericos, exceto whitespace
    df[column_name] = df[column_name].str.replace('[^\w\s]', ' ', regex=True)

    # Normalizar a string
    df[column_name] = df[column_name].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

    # Remover stopwords
    stop = ['DO', 'DA', 'DE', 'E']
    df[column_name] = df[column_name].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

    # Strip whitespace
    df[column_name] = df[column_name].apply(lambda x: x.strip())

    return df

In [34]:
# Lista de Curriculo Lattes

# Tratar os nomes da NOME-INSTITUICAO
df2 = df2.apply(lambda x: x.astype(str).str.upper())
df2 = clean_names(df2, 'NOME-INSTITUICAO')

# Dropar linhas com IES < 50
#df2 = df2[df2.groupby('NOME-INSTITUICAO')['NOME-INSTITUICAO'].transform('count').ge(50)]

# Split df: educacao & outros
df2_edu = df2.loc[df2['ENQUADRAMENTO-FUNCIONAL'].isin(['LIVRE', 'COLABORADOR', 'PROFESSOR_VISITANTE'])].copy(deep=True)
df2_outros = df2.loc[df2['ENQUADRAMENTO-FUNCIONAL'].isin(['SERVIDOR_PUBLICO', 'CELETISTA', 'NAN', 'OUTRO'])].copy(deep=True)

df2_edu

,NRO-ID-CNPQ,CODIGO-INSTITUICAO,NOME-INSTITUICAO,ANO-INICIO,ANO-FIM,ENQUADRAMENTO-FUNCIONAL,TIPO-DE-VINCULO,OUTRAS-INFORMACOES,CARGA-HORARIA-SEMANAL,FLAG-DEDICACAO-EXCLUSIVA
0,3397219421632200,000100000991,HOSPITAL UNIVERSITARIO PROFESSOR ALBERTO ANTUNES,2011,2015,LIVRE,LIVRE,ANALISTA JÚNIOR III,40.0,FALSE
1,3397219421632200,000100000991,HOSPITAL UNIVERSITARIO PROFESSOR ALBERTO ANTUNES,2016,2017,LIVRE,LIVRE,SECRETÁRIO EXECUTIVO,30.0,FALSE
2,3397219421632200,000200000993,SECRETARIA ESTADO EDUCACAO ALAGOAS,2014,2017,LIVRE,LIVRE,PROFESSOR MONITOR DA DISCIPLINA DE GEOGRAFIA,10.0,FALSE
4,3381149537136202,192900000005,UNIVERSIDADE ESTADO PARA,2009,2024,LIVRE,LIVRE,MONITORIA,15.0,TRUE
5,3366254034832265,J58200000003,EDITORA ABRIL,2013,2018,COLABORADOR,LIVRE,TRADUTOR INGLÊS / FRANCES,,FALSE
...,...,...,...,...,...,...,...,...,...,...
495,3310671206645194,000300000995,ETTORI VALINHOS,2018,2019,LIVRE,LIVRE,ASSISTENTE DE COMERCIO EXTERIOR,50.0,TRUE
496,3308359845841980,J5F100000007,CENTRO EDUCACIONAL MACUNAIMA,2012,2013,LIVRE,LIVRE,PROFESSOR DE INGLÊS,16.0,FALSE
497,3390386533322309,000400000997,ESCOLA MUNICIPAL,2007,2024,LIVRE,LIVRE,PROFESSORA,20.0,FALSE
498,3390386533322309,000500000999,JORNAL POMBO CORREIO,2009,2009,LIVRE,LIVRE,NAN,,FALSE


In [35]:
# Lista de IES
df_uni_filtered = df_uni_filtered.apply(lambda x: x.astype(str).str.upper())
df_uni_filtered = clean_names(df_uni_filtered, 'NOME_DA_IES')

# merge colunas NOME e SIGLA
df_uni_filtered['NOME_E_SIGLA'] = df_uni_filtered['SIGLA'] + ' ' + df_uni_filtered['NOME_DA_IES']
df_uni_filtered

,NOME_DA_IES,SIGLA,NOME_E_SIGLA
862,ICEV INSTITUTO ENSINO SUPERIOR,,ICEV INSTITUTO ENSINO SUPERIOR
2205,VOAS INSTITUTO APRENDIZAGEM SUPERIOR,VOAS,VOAS VOAS INSTITUTO APRENDIZAGEM SUPERIOR
2873,UNIVERSIDADE ESTADO MATO GROSSO CARLOS ALBERTO...,UNEMAT,UNEMAT UNIVERSIDADE ESTADO MATO GROSSO CARLOS ...
334,UNIVERSIDADE UNIVERSUS VERITAS GUARULHOS,UNIVERITAS UNG,UNIVERITAS UNG UNIVERSIDADE UNIVERSUS VERITAS ...
4124,UNIVERSIDADE REGIONAL CARIRI,,UNIVERSIDADE REGIONAL CARIRI
...,...,...,...
2416,ACADEMIA JUDICIAL TRIBUNAL JUSTICA SANTA CATARINA,AJ,AJ ACADEMIA JUDICIAL TRIBUNAL JUSTICA SANTA CA...
4234,ACADEMIA POLICIA MILITAR ESPIRITO SANTO,APM/ES,APM/ES ACADEMIA POLICIA MILITAR ESPIRITO SANTO
30,ACADEMIA POLICIA MILITAR MINAS GERAIS,APM,APM ACADEMIA POLICIA MILITAR MINAS GERAIS
3933,ACADEMIA FORCA AEREA,AFA,AFA ACADEMIA FORCA AEREA


In [38]:
# Split lista de IES por inicio do nome
df_uni_filtered.sort_values(by=["NOME_DA_IES"], ascending=True)

df_uni_outros = df_uni_filtered.copy(deep=True)

df_uni_instituto = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.startswith('INSTITUTO')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.startswith('INSTITUTO')].index, inplace=True)

df_uni_faculdades = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.startswith('FACULDADE')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.startswith('FACULDADE')].index, inplace=True)

df_uni_escola = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.startswith('ESCOLA')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.startswith('ESCOLA')].index, inplace=True)

df_uni_universidade = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.startswith('UNIVERSIDADE')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.startswith('UNIVERSIDADE')].index, inplace=True)

df_uni_instituicao = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.startswith('INSTITUICAO')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.startswith('INSTITUICAO')].index, inplace=True)

# List
# df_uni_outros, df_uni_instituto, df_uni_faculdades, df_uni_escola, df_uni_universidade, df_uni_instituicao

## NLP


### Cosine

In [9]:
def iterate_sigla(nome_instituicao, df_for_search):

    matched_rows = []

    print(nome_instituicao)

    for index, row in df_for_search.iterrows():

        if row['SIGLA']:

            pattern = r'\b' + row['SIGLA'] + r'\b'
            if re.search(pattern, nome_instituicao):
                matched_rows.append(row['SIGLA'])
                print(row['SIGLA'], row['NOME_DA_IES'])

            # dos diversos matches, qual mais faz sentido?
            #if len(matched_rows) > 1:


    print("============")

    return ",".join(str(element) for element in matched_rows)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

threshold = 0.8

# Vetorização TF-IDF
vectorizer = TfidfVectorizer().fit(df_uni_filtered['NOME_DA_IES'])
tfidf_matrix_universidades = vectorizer.transform(df_uni_filtered['NOME_DA_IES'])

df2_edu['nome_corrigido'] = ''
df2_edu['sigla_detectado'] = ''

for index, row in df2_edu.iterrows():

    nome_instituicao = row['NOME-INSTITUICAO']

    if (nome_instituicao.startswiwth())
        df_to_search =

    # TODO: Ver se contém a sigla
    sigla_detectado = iterate_sigla(nome_instituicao, )
    if sigla_detectado:
        df2_edu['sigla_detectado'] = sigla_detectado

    else:
        vector_comparacao = vectorizer.transform([nome_instituicao])

        similaridade = cosine_similarity(vector_comparacao, tfidf_matrix_universidades)

        max_value = np.max(similaridade)
        max_index = np.argmax(similaridade)

        nome_corrigido = df_uni_filtered.iloc[max_index]['NOME_DA_IES']

        if (max_value >= threshold):
            df2_edu.at[index, 'nome_corrigido'] = nome_corrigido
        else:
            df2_edu.at[index, 'nome_corrigido'] = ''

HOSPITAL UNIVERSITARIO PROFESSOR ALBERTO ANTUNES
HOSPITAL UNIVERSITARIO PROFESSOR ALBERTO ANTUNES
SECRETARIA ESTADO EDUCACAO ALAGOAS
UNIVERSIDADE ESTADO PARA
EDITORA ABRIL
UNYLEYA EDITORA CURSOS S A
CENTRO UNIVERSITARIO FAMETRO
FAMETRO FACULDADE METROPOLITANA PARA
FAMETRO FACULDADE METROPOLITANA TEFE
FAMETRO FACULDADE METROPOLITANA PARINTINS
FAMETRO FACULDADE METROPOLITANA ITACOATIARA
FAMETRO FACULDADE METROPOLITANA COARI
FAMETRO CENTRO UNIVERSITARIO CEUNI FAMETRO
UNIAO ESCOLAS SUPERIORES RONDONOPOLIS
GOVERNO ESTADO MINAS GERAIS
ENDOIMAGEM SERVICOS MEDICOS LTDA
AMIU BOTAFOGO
UNIVERSIDADE PAULISTA
MARTINS COMERCIO SERVICOS DISTRIBUICAO S A
SUPERMERCADO BRASIL NORTE LTDA
COMERCIAL REAL CARNES LTDA
LANCHONETE SORVETERIA PISTACHIOS
MEISTER S A
DEPARTAMENTO MUNICIPAL AGUA ESGOTO UBERLANDIA
SORVETES LIDER
CONTA FACIL PRESTACAO SERVICOS LTDA
DROGA RAIA S A
MEIZLER BIOPHARMA LTDA
DARROW LABORATORIOS
MOKSHA8 REPRESENTACOES
SANOFI AVENTIS FARMACEUTICA
LABORATORIO QUIMICO FARMACEUTICO BERGAMO
FAC

In [ ]:
# oq foi alterado
pd.set_option('display.max_rows', 10)
print(df2_edu[['NOME-INSTITUICAO', 'nome_corrigido']].loc[df2_edu['NOME-INSTITUICAO'] != df2_edu['nome_corrigido']])
pd.set_option('display.max_rows', 6)

In [ ]:
df2_edu.loc[df2_edu['sigla_detectado'] == True]

,NRO-ID-CNPQ,CODIGO-INSTITUICAO,NOME-INSTITUICAO,ANO-INICIO,ANO-FIM,ENQUADRAMENTO-FUNCIONAL,TIPO-DE-VINCULO,OUTRAS-INFORMACOES,CARGA-HORARIA-SEMANAL,FLAG-DEDICACAO-EXCLUSIVA,nome_corrigido,sigla_detectado


### Cosine by N-gram


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

threshold = 0.8

vectorizer = TfidfVectorizer(ngram_range=(2, 3)).fit(df_uni_filtered['NOME_DA_IES'])
tfidf_matrix_universidades = vectorizer.transform(df_uni_filtered['NOME_DA_IES'])

df2_edu['nome_corrigido_ngram'] = ''

for index, row in df2_edu.iterrows():
    nome_instituicao = row['NOME-INSTITUICAO']

    vector_comparacao = vectorizer.transform([nome_instituicao])

    similaridade = cosine_similarity(vector_comparacao, tfidf_matrix_universidades)

    max_value = np.max(similaridade)
    max_index = np.argmax(similaridade)

    nome_corrigido = df_uni_filtered.iloc[max_index]['NOME_DA_IES']

    if max_value >= threshold:
        df2_edu.at[index, 'nome_corrigido_ngram'] = nome_corrigido
    else:
        df2_edu.at[index, 'nome_corrigido_ngram'] = nome_instituicao

In [ ]:
pd.set_option('display.max_rows', 100)
print(df2_edu[['NOME-INSTITUICAO', 'nome_corrigido_ngram']].loc[df2_edu['NOME-INSTITUICAO'] != df2_edu['nome_corrigido_ngram']])

                                      NOME-INSTITUICAO  \
38   UNIVERSIDADE FEDERAL PIAUI CAMPUS MINISTRO REI...   
39   UNIVERSIDADE FEDERAL PIAUI CAMPUS MINISTRO REI...   
40   UNIVERSIDADE FEDERAL PIAUI CAMPUS MINISTRO REI...   
41   UNIVERSIDADE FEDERAL PIAUI CAMPUS MINISTRO REI...   
42   UNIVERSIDADE FEDERAL PIAUI CAMPUS MINISTRO REI...   
43   UNIVERSIDADE FEDERAL PIAUI CAMPUS MINISTRO REI...   
44   UNIVERSIDADE FEDERAL PIAUI CAMPUS MINISTRO REI...   
187  INSTITUTO PSIQUIATRIA HOSPITAL DAS CLINICAS FA...   
283                INSTITUTO CENTRO ENSINO TECNOLOGICO   
295                 HOSPITAL UNIVERSITARIO SANTA MARIA   
342              FACULDADE PIO DECIMO CAMPUS JABUTIANA   
343              FACULDADE PIO DECIMO CAMPUS JABUTIANA   
344              FACULDADE PIO DECIMO CAMPUS JABUTIANA   
350  LIGA ORTOPEDIA TRAUMATOLOGIA UNIVERSIDADE CATO...   
351  LIGA ORTOPEDIA TRAUMATOLOGIA UNIVERSIDADE CATO...   
365            UNIVERSIDADE FEDERAL VALE SAO FRANCISCO   
374           